**postgis+geoserver+openlayer实现最短路径分析**

**软件环境**

操作系统：Win10 pro

数据库：postgreSQL9.5 + postgis2.5

Gis桌面端：Arcgis10.7，Udig2.0

版本选择原因：

PostgreSQL16版本对于uDig和Arcgis10.7版本太高，兼容这两个软件，选择安装PG9.5

Arcgis10.8支持pg9.6以上，pg9.6在当前的Win10 pro上安装时报错，vc++环境安装不上

**名词**

**PostgreSQL**：是一个开源的关系型数据库管理系统（RDBMS），它以其高度的可扩展性、可靠性和对SQL标准的支持而闻名。

**PostGIS**：是一个基于PostgreSQL数据库的空间数据库扩展，为地理信息系统提供了坚实的数据存储和处理能力，是GIS专业人士和开发者在进行空间数据分析时的常用工具之一。

**GeoServer** : GeoServer是一个服务器软件，可以发布地图数据到Web上，支持多种数据源和多种地图服务标准。

**OpenLayers** : OpenLayers是一个广泛使用的开源JavaScript库，专门用于创建交互式的Web地图，它支持多种地图服务接口。

**uDig** 是一个开源桌面地理信息系统（Desktop GIS）。是基于Eclipse Rich Client Platform (RCP) 和GeoTools开发的，它支持OpenGIS组织发布的公共标准，尤其支持Web地图服务器（WMS）和Web功能服务器（WFS）标准。它既可以作为一个独立的应用程序使用，也可以作为开发新的桌面因特网GIS应用程序的框架。uDig提供了一系列的功能，包括地图查看、编辑、空间数据分析等，适用于需要处理地理空间数据的用户。

**参考**

[PostGIS 导入SHP文件并与ArcGIS连接](https://blog.csdn.net/guzicheng1990/article/details/99820683?spm=1001.2014.3001.5502)

[PostGIS 结合Openlayers以及Geoserver实现最短路径分析（一）](https://blog.csdn.net/guzicheng1990/article/details/102524923?spm=1001.2014.3001.5502)

pgRouting windows下，安装PostGis时，自动安装pgRouting

[PostGIS 结合Openlayers以及Geoserver实现最短路径分析（二）](https://blog.csdn.net/guzicheng1990/article/details/102524953?spm=1001.2014.3001.5502)

[PostGIS 结合Openlayers以及Geoserver实现最短路径分析（三）](https://blog.csdn.net/guzicheng1990/article/details/102525013?spm=1001.2014.3001.5502)

[基于PgRouting的GIS网络分析--数据准备](https://www.jianshu.com/p/34c8378c3da9)
[PostGIS管网连通性分析](https://blog.csdn.net/gisarmory/article/details/116194268)

[PostGIS/pgRouting 最优路径规划 管网连通性分析](https://www.cnblogs.com/xieshier/p/16673734.html)

[官网网络分析数据](https://docs.pgrouting.org/3.0/en/sampledata.html)

**软件环境**

postgreSQL9.5

postgis2.5

geoServer安装

Udig2.0

**流程**

用arcmap生成线路交点打断的路线数据，导入postgis数据库中，进行拓扑处理。基于postgis提供的分析函数编写最短路径分析函数。在geoserver中发布底图和sql查询视图，openlayer调用。

**ArcMap数据处理**

在arcmap中，简单画一个如下图的路网，然后将相交的线在交点处打断（打开Advanced Editing工具，用其Line Intersection功能，将线两两打断）。

![示例路网](img/shp01.png)

新建一个Id,Id=Fid+1（原因导入postgis后Fid变成gid，gid的值=fid+1）。

导入postgis

使用postgis自带的工具“PostGIS 2.0 Shapefile and DBF Loader Exporter”导入数据

连接数据库PostGis数据源

![连接数据库](img/import_conn.png)

添加shp文件，①设置坐标系sird值，②字符集，如果有汉字时将utf-8设置成GBK或gb18030（shp文件存放的路径不要有中文）。③选择生成简单几何图形。导入post数据库。

![连接数据库](img/import_set.png)


In [ ]:
**数据预处理**

1、打开pgAdmin Ⅲ，运行如下预处理语句

--添加起点字段source
ALTER TABLE zy ADD COLUMN source integer;

--添加终点字段target
ALTER table zy add column target integer;

--添加道路权重值字段
ALTER TABLE zy ADD COLUMN length double precision;

--建立拓扑关系，这里会为source和target字段赋值（表明，阈值，要素字段名，主键id）
SELECT pgr_createTopology('zy', 0.00001, 'geom', 'gid');

--为source和target字段创建索引（加快查询速度）
CREATE INDEX source_idx ON zy("source");
CREATE INDEX target_idx ON zy("target");

--添加线段端点坐标
ALTER TABLE ZY ADD COLUMN x1 double precision;        --创建起点经度x1
ALTER TABLE ZY ADD COLUMN y1 double precision;        --创建起点纬度y1
ALTER TABLE ZY ADD COLUMN x2 double precision;        --创建起点经度x2
ALTER TABLE ZY ADD COLUMN y2 double precision;        --创建起点经度y2

--给x1、y1、x2、y2赋值
UPDATE ZY SET x1 =ST_x(ST_PointN(geom, 1));    
UPDATE ZY SET y1 =ST_y(ST_PointN(geom, 1));    
UPDATE ZY SET x2 =ST_x(ST_PointN(geom, ST_NumPoints(geom)));    
UPDATE ZY SET y2 =ST_y(ST_PointN(geom, ST_NumPoints(geom)));     

--为length赋值（以长度作为权重）
UPDATE zy SET length = st_length(geom);

--将长度值赋给reverse_cost，作为路线选择标准（消耗）
ALTER TABLE zy ADD COLUMN reverse_cost double precision;
UPDATE zy SET reverse_cost = st_length(geom);


2、测试数据，如果执行下面的语句有结果，则表示数据预处理成功

In [ ]:
--通过起点号、终点号查询最短路径
--source为线表起点字段名称
--target为线表终点字段名称
--起点终点前后顺序无固定要求
--length为长度字段，也可以使用自己的评价体系
--1、9为测试使用起点号\终点号
--zy表名
--id1经过节点号
--id2经过路网线的gid
SELECT seq, id1 AS node, id2 AS edge, cost FROM pgr_dijkstra('
                SELECT  gid AS id,
                         source::integer,
                         target::integer,
                         length::double precision AS cost
                        FROM zy',
                1, 4, false, false);

3、创建最短路径查询函数（核心）

In [ ]:
-- Function: public.pgr_shortestpath(character varying, double precision, double precision, double precision, double precision, integer)
--DROP FUNCTION public.pgr_shortestpath(character varying, double precision, double precision, double precision, double precision, integer);

CREATE OR REPLACE FUNCTION public.pgr_shortestpath(
    tbl character varying,
    startx double precision,
    starty double precision,
    endx double precision,
    endy double precision,
    srid  integer DEFAULT 3857)
  RETURNS geometry AS
$BODY$  
 
declare  
    v_startLine geometry;--离起点最近的线 
    v_endLine geometry;--离终点最近的线  
    v_startTarget integer;--距离起点最近线的终点 
    v_startSource integer; 
    v_endSource integer;--距离终点最近线的起点 
    v_endTarget integer;
    v_statpoint geometry;--在v_startLine上距离起点最近的点  
    v_endpoint geometry;--在v_endLine上距离终点最近的点  
    v_res geometry;--最短路径分析结果 
    v_res_a geometry; 
    v_res_b geometry; 
    v_res_c geometry; 
    v_res_d geometry;  
    v_perStart float;--v_statpoint在v_res上的百分比  
    v_perEnd float;--v_endpoint在v_res上的百分比  
    v_shPath_se geometry;--开始到结束 
    v_shPath_es geometry;--结束到开始 
    v_shPath geometry;--最终结果 
    tempnode float;      
begin 
    --查询离起点最近的线 
    --3857坐标系
    --找起点15米范围内的最近线 
    execute 'select geom, source, target  from ' ||tbl|| 
                            ' where ST_DWithin(geom,ST_GeometryFromText(''point('||startx ||' ' || starty||')'',' ||srid|| '),200)
                            order by ST_Distance(geom,ST_GeometryFromText(''point('|| startx ||' '|| starty ||')'',' ||srid|| '))  limit 1' 
                            into v_startLine, v_startSource ,v_startTarget;  
    --RAISE NOTICE '查询离起点最近的线: ';                        
    --RAISE NOTICE 'v_startLine: %', v_startLine;
    --RAISE NOTICE 'v_startSource: %', v_startSource;
    --RAISE NOTICE 'v_startTarget: %', v_startTarget;
    --查询离终点最近的线 
    --找终点15米范围内的最近线 
    execute 'select geom, source, target from ' ||tbl|| 
                            ' where ST_DWithin(geom,ST_GeometryFromText(''point('|| endx || ' ' || endy ||')'',' ||srid|| '),200)
                            order by ST_Distance(geom,ST_GeometryFromText(''point('|| endx ||' ' || endy ||')'',' ||srid|| '))  limit 1' 
                            into v_endLine, v_endSource,v_endTarget;  
    RAISE NOTICE '查询离终点最近的线: ';
    RAISE NOTICE 'v_endLine: %', v_endLine;
    RAISE NOTICE 'v_endSource: %', v_endSource;
    RAISE NOTICE 'v_endTarget: %', v_endTarget;
    --如果没找到最近的线，就返回null  
    if (v_startLine is null) or (v_endLine is null) then  
        return null;  
    end if ;  
    --找到最近的点
    select  ST_ClosestPoint(v_startLine, ST_GeometryFromText('point('|| startx ||' ' || starty ||')',srid)) into v_statpoint;  
    select  ST_ClosestPoint(v_endLine, ST_GeometryFromText('point('|| endx ||' ' || endy ||')',srid)) into v_endpoint;  
    RAISE NOTICE '找到最近的点: ';
    RAISE NOTICE 'v_endSource: %', v_endSource;
    RAISE NOTICE 'v_endTarget: %', v_endTarget;
   -- ST_Distance  （下面是否可以直接用pgr_dijkstra函数，不做端点判断，待研究）
    --从开始的起点到结束的起点最短路径 
    execute 'SELECT st_linemerge(st_union(b.geom)) ' ||
    'FROM pgr_kdijkstraPath( 
    ''SELECT gid as id, source, target, length as cost FROM ' || tbl ||''','  
    ||v_startSource|| ', ' ||'array['||v_endSource||'] , false, false 
    ) a, '  
    ||tbl|| ' b 
    WHERE a.id3=b.gid   
    GROUP by id1   
    ORDER by id1' into v_res ;
    --从开始的终点到结束的起点最短路径 
    execute 'SELECT st_linemerge(st_union(b.geom)) ' || 
    'FROM pgr_kdijkstraPath( 
    ''SELECT gid as id, source, target, length as cost FROM ' || tbl ||''','  
    ||v_startTarget|| ', ' ||'array['||v_endSource||'] , false, false 
    ) a, '  
    ||tbl|| ' b 
    WHERE a.id3=b.gid   
    GROUP by id1   
    ORDER by id1' into v_res_b ; 
    --从开始的起点到结束的终点最短路径 
    execute 'SELECT st_linemerge(st_union(b.geom)) ' || 
    'FROM pgr_kdijkstraPath( 
    ''SELECT gid as id, source, target, length as cost FROM ' || tbl ||''','  
    ||v_startSource || ', ' ||'array['||v_endTarget||'] , false, false 
    ) a, '  
    || tbl || ' b 
    WHERE a.id3=b.gid   
    GROUP by id1   
    ORDER by id1' into v_res_c ; 
    --从开始的终点到结束的终点最短路径 
    execute 'SELECT st_linemerge(st_union(b.geom)) ' || 
    'FROM pgr_kdijkstraPath( 
    ''SELECT gid as id, source, target, length as cost FROM ' || tbl ||''','  
    ||v_startTarget || ', ' ||'array['||v_endTarget||'] , false, false 
    ) a, '  
    || tbl || ' b 
    WHERE a.id3=b.gid   
    GROUP by id1   
    ORDER by id1' into v_res_d ; 
    if(ST_Length(v_res) > ST_Length(v_res_b)) then 
       v_res = v_res_b; 
    end if; 
    if(ST_Length(v_res) > ST_Length(v_res_c)) then 
       v_res = v_res_c; 
    end if; 
    if(ST_Length(v_res) > ST_Length(v_res_d)) then 
       v_res = v_res_d; 
    end if; 
    --如果找不到最短路径，就返回null  
    --if(v_res is null) then  
    --    return null;  
    --end if;  
    --将v_res,v_startLine,v_endLine进行拼接  select  ST_LineMerge(ST_Union(array[v_res,v_startLine,v_endLine])) into v_res; 
    raise notice '%',  v_res; 
    select  ST_Line_Locate_Point(v_res, v_statpoint) into v_perStart;  
    select  ST_Line_Locate_Point(v_res, v_endpoint) into v_perEnd;  
    if(v_perStart > v_perEnd) then  
        tempnode =  v_perStart; 
        v_perStart = v_perEnd; 
        v_perEnd = tempnode; 
    end if; 
    --截取v_res 
    --拼接线 
    SELECT ST_Line_SubString(v_res,v_perStart, v_perEnd) into v_shPath; 
    raise notice '%',  v_shPath; 
    return v_shPath;  
end;  
$BODY$
  LANGUAGE plpgsql VOLATILE STRICT
  COST 100;
ALTER FUNCTION public.pgr_shortestpath(character varying, double precision, double precision, double precision, double precision,character varying)
  OWNER TO postgres;


4、测试，刚开始时怎么运行都没结果，以为代码错了，后来一点点试验，起点终点坐标从数据表里找了两个就有结果了，可能是我测试数据的范围太大了，导致指定范围内差不多数据。

测试语句

SELECT * FROM pgr_shortestpath('zy', 13722126.6941279,5036398.09328714,13693143.583109,4995007.75154479,3857)

5、在创建拓扑时，有时候会遇到数据量非常大的情况，一次执行会非常消耗内存，这里建议分批创建

In [ ]:
##推荐分批拓扑

select pgr_createTopology('zy',0.1,source:='source',id:='gid',target:='target',the_geom:='geom',rows_where:='gid < 50000')
 
select pgr_createTopology('zy',0.1,source:='source',id:='gid',target:='target',the_geom:='geom',rows_where:='gid >= 50000 and gid < 100000')
 
select pgr_createTopology('zy',0.1,source:='source',id:='gid',target:='target',the_geom:='geom',rows_where:='gid >= 100000 and gid < 150000')
 
select pgr_createTopology('zy',0.1,source:='source',id:='gid',target:='target',the_geom:='geom',rows_where:='gid >= 150000 and gid < 200000')
 
select pgr_createTopology('zy',0.1,source:='source',id:='gid',target:='target',the_geom:='geom',rows_where:='gid >= 200000 and gid < 250000')
 
select pgr_createTopology('zy',0.1,source:='source',id:='gid',target:='target',the_geom:='geom',rows_where:='gid >= 250000 and gid < 300000')
 
select pgr_createTopology('zy',0.1,source:='source',id:='gid',target:='target',the_geom:='geom',rows_where:='gid >= 300000 and gid < 350000')
 
select pgr_createTopology('zy',0.1,source:='source',id:='gid',target:='target',the_geom:='geom',rows_where:='gid >= 350000 and gid < 450000')

**发布数据**

1、创建工作区

![示例路网](img/creat_workspaces.png)

2、创建数据存储

![新建数据存储](img/creat_stores.png)

填写相关参数

![新建数据存储](img/creat_stores2.png)

3、创建图层

创建两类图层，一是底图，一个是sql视图。④创建是地图，⑤是创建sql视图

![新建数据存储](img/creat_layer.png)

创建sql视图

sql 视图：

SELECT * FROM pgr_shortestpath(‘zy’, %x1%, %y1%, %x2%, %y2%)

验证的正则表达式：^-?[\d.]+$

类型：LingString

SRID：3857

![新建数据存储](img/creat_sqlView.png)


**openlayer 调用**


In [ ]:
<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8" />
    <title>网络分析</title>
    <link href="http://localhost:8080/test/ol.css" rel="stylesheet" type="text/css">
    <style>
        #id {
            width: 1000px;
            height: 1000px;
        }
    </style>
    <script type="text/javascript" src="http://localhost:8080/test/ol.js"></script>
</head>

<body>
    <div id="map"></div>
    <script>

        var roadLayer = new ol.layer.Tile({
            source: new ol.source.TileWMS({
                url: 'http://localhost:8080/geoserver/NA/wms',
                params: { 'LAYERS': 'NA:zy', 'TILED': true },
                serverType: 'geoserver'
            })
        })
        var map = new ol.Map({
            target: document.getElementById("map"),
            layers: [
                roadLayer
            ],
            view: new ol.View({
                center: [13700000, 5015000],
                projection: 'EPSG:3857',
                zoom:10
            })
        });
        var startCoord = [13722126.6941279,5036398.09328714];
      var destCoord = [13693143.583109,4995007.75154479];
        var params = {
            LAYERS: 'NA:func_shortestpath',
            FORMAT: 'image/png',
        };
        var viewparams = [
            'x1:' + startCoord[0], 'y1:' + startCoord[1],
            'x2:' + destCoord[0], 'y2:' + destCoord[1]
            //'x1:' + 12952117.2529, 'y1:' + 4836395.5717,
            //'x2:' + 12945377.2585, 'y2:' + 4827305.7549
        ];
        console.log(viewparams);
        params.viewparams = viewparams.join(';');
        result = new ol.layer.Image({
            source: new ol.source.ImageWMS(
                {
                    url: 'http://localhost:8080/geoserver/NA/wms',
                    params: params
                })
        });
        map.addLayer(result);
    </script>
</body>

</html>

效果

![效果](img/result.png)